In [1]:
import torch
import cv2
import numpy as np
from PIL import Image
import torch.nn as nn
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F


In [2]:
DATA_DIR = "./other_images/"
FILE_NAME = "000468_0930221420554_037_RGB1_9.jpg"

image_path = DATA_DIR + FILE_NAME

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)

IMAGE_SIZE = 224


cpu


In [3]:
net = models.resnet18()
net.fc = nn.Linear(net.fc.in_features, 2)

net.load_state_dict(torch.load("./best_model_resnet18.pt", map_location=DEVICE))
finalconv_name = 'layer4'
net.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
features_blobs = []

def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

# print(features_blobs)

In [6]:
net._modules.get(finalconv_name).register_forward_hook(hook_feature)

# print(features_blobs) 

[]


In [7]:
params = list(net.parameters()) # 将参数变换为列表
weight_softmax = np.squeeze(params[-2].data.numpy()) # 提取softmax层的参数
print(weight_softmax.shape, features_blobs)

(2, 512) []


In [8]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bc, nc, h, w = feature_conv.shape
    output_cam = []
    # class_idx为预测分数较大的类别数字表示的数组，一张图片中有N个类物体，则数组中N个元素
    for idx in class_idx:
        # 回到GAP的值
        # weight_softmax中预测为第idx类的参数w乘以feature_map,为了相乘reshape map的形状
        cam = weight_softmax[idx].dot(feature_conv.reshape(nc, h * w))
        #将feature_map的形状reshape回去
        cam = cam.reshape(h, w)
				
				# 归一化操作（最小值为0，最大值为1）
        # np.min 返回数组的最小值或沿轴的最小值
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
			
				# 转换为图片255的数据
        # np.uint8() Create a data type object.
        cam_img = np.uint8(255 * cam_img)

			 # resize 图片尺寸与输入图片一致
        output_cam.append(cv2.resize(cam_img, size_upsample))

    return output_cam

In [9]:
# 数据处理，先缩放尺寸到(224,224)，再变换数据类型为tensor,最后normalize
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    normalize
])

In [10]:
img_pil = Image.open(image_path)
img_pil.save('test.jpg')
# 将图片数据处理成所需要的可用数据 tensor
img_tensor = preprocess(img_pil)
# 处理图片为Variable数据
img_variable = Variable(img_tensor.unsqueeze(0))
#将图片输入网络得到预测类别分数
logit = net(img_variable)
# np.shape(features_blobs)

In [13]:
h_x = F.softmax(logit, dim=1).data.squeeze()
print(h_x)
# 对分类的预测类别分数排序，输出预测值和在列表中的位置
idx = 1
probs = h_x[1]

# 转换数据类型
probs = probs.numpy()
idx = np.array(idx)

print(probs, idx)
print(type(probs), type(idx))
print(probs.shape, idx.shape)

tensor([2.0474e-04, 9.9980e-01])
0.99979526 1
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
() ()


In [14]:
# 输出与图片尺寸一致的CAM图片
# generate class activation mapping for the top1 prediction
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx])

In [ ]:
# render the CAM and output
print("output CAM.jpg")

img = cv2.imread(image_path)
height, width, _ = img.shape
# 生成热力图
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0], (width, height)), cv2.COLORMAP_JET)
result = cv2.addWeighted(img, 0.3, heatmap, 0.5, 0)
cv2.imwrite('CAM.jpg', result)
cv2.imshow("heatmap", result)